In [ ]:
import pandas as pd
import os 
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm

from joblib import Parallel, delayed

In [ ]:
# Seleccionar que años se van a tomar en cuenta, discutir esto
years = [str(i) for i in range(1995, 2024)]

datasets = []
for year in years:
    path = os.path.join("dataRaw", "atp_matches_" + year)
    dset = pd.read_csv(path + ".csv")
    datasets.append(dset)

In [ ]:
df = pd.concat(datasets)
df = df.reset_index(drop = True)
df

In [ ]:
# Selección de features
df.isnull().sum()/len(df)

In [ ]:
# Definir técnicas para rellenar datos, pero manteniendo una versión con datos clean.


In [ ]:
# Viendo el año, podemos construir un set de testing, el siguiente número es el procentaje
# del tamaño del testing en comparación al total.

len(df[df["tourney_date"] < 20150101])/len(df)

In [ ]:
len(df[(df["tourney_date"] >= 20150101) & (df["tourney_date"] <= 20181231)])/len(df)

In [ ]:
len(df[df["tourney_date"] > 20190101])/len(df)

### Que hacer para el avance 

- Decidir un espacio de tiempo, junto a que features a utilizar y como tratar los datos faltantes. XXX
- Tratar features cualitativos. XXX
- Detección de features altamente correlacionados.
- Detección de outliers.
- Definición del set de testeo, por posible problema de dependencia temporal.
- Selección de modelos de clasificación a probar para predicción de resultados.

### Definir el dataset

- Se usará los partidos del 2000 a 2017 como training.
- Se usaran los datos de 2024 como validación.
- Se usará los partidos del 2018 a 2023 como testing.


In [ ]:
df.isnull().sum()/len(df)

### No eliminar data, puesto que vamos a usar promedios.

In [ ]:
cuantitative_variables_names = df.select_dtypes(["float","int"]).columns.tolist()
df_corr = df[cuantitative_variables_names].corr()
sns.heatmap(df_corr, cmap = "rocket")

In [ ]:
# Features a usar

featuresNoEntrenables = ["winner_seed", "winner_entry", "loser_seed", "loser_entry"
                       "minutes", "round", "score", "tourney_id", "tourney_name"
                       "draw_size", "tourney_level", "tourney_date", "match_num",
                       "winner_name","loser_name"]

featuresDelMatch = ["surface", "winner_hand", "loser_hand", "winner_age",
                    "loser_age", "best_of","tourney_date"]

# best_of debe ser categorico

# featuresDeMatchPasados = #NUMERICOS


In [ ]:
### Recopilar información de los n partidos hacia atras de cada jugador.
def pastInfo(df, num_matches, i):

    # Para la búsqueda de features
    prefixes = ['winner_', "w_"]
    filtered_columns_winner = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]

    prefixes = ['loser_', "l_"]
    filtered_columns_loser = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    
    # Solo partidos del 2000 en adelante
    if df.iloc[i]["tourney_date"] < 20000101:
        return pd.DataFrame()
    winner_name = df.iloc[i].winner_name
    loser_name = df.iloc[i].loser_name

    # Partidos del ganador

    dfAux = deepcopy(df.iloc[:i])
    dfAux = dfAux[(dfAux.loser_name == winner_name)| (dfAux.winner_name == winner_name)].sort_values(by=["tourney_date"], ignore_index=True)[-num_matches:]
    dataWinner = []
    for j in range(len(dfAux)):

        if dfAux.iloc[j].winner_name == winner_name:
            filterDrop = dfAux.iloc[j][filtered_columns_winner].drop(["winner_hand","winner_age", "winner_ioc", "winner_name","winner_entry","winner_id","winner_seed"], axis = 0)
            data = pd.DataFrame(filterDrop).T
            newColumns = data.columns.str.replace(r'^(winner_|w_)', 'player1_', regex=True)
            data.columns = newColumns
            dataWinner.append(data)

        elif dfAux.iloc[j].loser_name == winner_name:
            filterDrop = dfAux.iloc[j][filtered_columns_loser].drop(["loser_hand","loser_age", "loser_ioc", "loser_name","loser_entry","loser_id","loser_seed"], axis = 0)
            data = pd.DataFrame(filterDrop).T
            newColumns = data.columns.str.replace(r'^(loser_|l_)', 'player1_', regex=True)
            data.columns = newColumns
            dataWinner.append(data)

    # Partidos del perdedor

    dfAux = deepcopy(df.iloc[:i])
    dfAux = dfAux[(dfAux.loser_name == loser_name)| (dfAux.winner_name == loser_name)].sort_values(by=["tourney_date"], ignore_index=True)[-num_matches:]
    dataLoser = []
    for j in range(len(dfAux)):
        
        if dfAux.iloc[j].winner_name == loser_name:
            filterDrop = dfAux.iloc[j][filtered_columns_winner].drop(["winner_hand","winner_age", "winner_ioc", "winner_name","winner_entry","winner_id","winner_seed"], axis = 0)
            data = pd.DataFrame(filterDrop).T
            newColumns = data.columns.str.replace(r'^(winner_|w_)', 'player2_', regex=True)
            data.columns = newColumns
            dataLoser.append(data)

        elif dfAux.iloc[j].loser_name == loser_name:
            filterDrop = dfAux.iloc[j][filtered_columns_loser].drop(["loser_hand","loser_age", "loser_ioc", "loser_name","loser_entry","loser_id","loser_seed"], axis = 0)
            data = pd.DataFrame(filterDrop).T
            newColumns = data.columns.str.replace(r'^(loser_|l_)', 'player2_', regex=True)
            data.columns = newColumns
            dataLoser.append(data)
    # Si no hay información suficiente del partido, pasar  OJO DECIDIR ESTO BIEN DESPUES################
    if len(dataWinner) == 0 or len(dataLoser) == 0:
        return pd.DataFrame()
    dataWinner = pd.concat(dataWinner).mean().to_frame().T
    dataLoser = pd.concat(dataLoser).mean().to_frame().T


    dfAux = deepcopy(df.iloc[:i])
    dfAuxWinnerWin = dfAux[(dfAux.winner_name == winner_name)]
    dfAuxWinnerLoss = dfAux[(dfAux.loser_name == winner_name)]
    dfAuxLoserWin = dfAux[(dfAux.winner_name == loser_name)]
    dfAuxLoserLoss = dfAux[(dfAux.loser_name == loser_name)]

    # Partidos ganados/perdidos por tipo de suelo
    superficies = df["surface"].unique()
    for suelo in superficies:
        dataWinner[f"player1_Wins_On_{suelo}"] = len(dfAuxWinnerWin[dfAuxWinnerWin.surface == suelo])
        dataWinner[f"player1_Loss_On_{suelo}"] = len(dfAuxWinnerLoss[(dfAuxWinnerLoss.surface == suelo)])
        dataLoser[f"player2_Wins_On_{suelo}"] = len(dfAuxLoserWin[dfAuxLoserWin.surface == suelo])
        dataLoser[f"player2_Loss_On_{suelo}"] = len(dfAuxLoserLoss[(dfAuxLoserLoss.surface == suelo)])
    
    # Partidos ganados/perdidos por tipo de mano
    manos = df["winner_hand"].unique()
    for mano in manos:
        dataWinner[f"player1_Wins_On_{mano}"] = len(dfAuxWinnerWin[dfAuxWinnerWin.loser_hand == mano])
        dataWinner[f"player1_Loss_On_{mano}"] = len(dfAuxWinnerLoss[(dfAuxWinnerLoss.winner_hand == mano)])
        dataLoser[f"player2_Wins_On_{mano}"] = len(dfAuxLoserWin[dfAuxLoserWin.loser_hand == mano])
        dataLoser[f"player2_Loss_On_{mano}"] = len(dfAuxLoserLoss[(dfAuxLoserLoss.winner_hand == mano)])

    dataMatch = pd.DataFrame(df.iloc[i][featuresDelMatch]).T.reset_index(drop=True)
    dataMatch.rename(columns={'winner_hand': 'player1_hand', "loser_hand": "player2_hand",
                            "winner_age":"player1_age", "loser_age": "player2_age"}, inplace=True)
    
    dataFinal = pd.concat([dataMatch, dataWinner, dataLoser], axis = 1)
    dataFinal["label"] = 0
    dataFinal["index"] = i # Para poder identificar los datasets
    return dataFinal

In [ ]:
# Búsqueda serial
nums = [15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
for num_matches in nums:
    resultado = []
    for i in tqdm(range(len(df))):
        resultado.append(pastInfo(df, num_matches, i))
    dfTotal = pd.concat(resultado, axis = 0)
    dfTotal.to_csv(f"Datasets\{num_matches}MeanPastMatches.csv")

In [ ]:
# Duplicarlos:
nums = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
for num_matches in nums:
    file = pd.read_csv(f"Datasets/{num_matches}MeanPastMatches.csv")
    file = file.drop("Unnamed: 0", axis = 1)
    # file.rename(columns={"player2_hand.1":"player2_age"}, inplace=True)

    new_file = pd.DataFrame()

    # Copiar la columna surface, la fecha, el label, la duración y el index para identificarlo.
    new_file["surface"] = file["surface"]
    new_file["tourney_date"] = file["tourney_date"]
    new_file["label"] = 1
    new_file["best_of"] = file["best_of"]
    new_file["index"] = file["index"]

    columnas_intercambiables = ["hand","age", "ht", "ace", "df", "svpt", "1stIn", "1stWon", "2ndWon", "SvGms", "bpSaved", "bpFaced", "rank", "rank_points"]

    superficies = df["surface"].unique()
    manos = df["winner_hand"].unique()
    
    for suelo in superficies:
        columnas_intercambiables.append(f"Wins_On_{suelo}")
        columnas_intercambiables.append(f"Loss_On_{suelo}")
    for mano in manos:
        columnas_intercambiables.append(f"Wins_On_{mano}")
        columnas_intercambiables.append(f"Loss_On_{mano}")
        
    
    for columna in columnas_intercambiables:
        nueva_columna_player1 = "player1_" + columna
        nueva_columna_player2 = "player2_" + columna
        new_file[nueva_columna_player2] = file[nueva_columna_player1]
        new_file[nueva_columna_player1] = file[nueva_columna_player2]

    # Si necesitas guardar el nuevo DataFrame en un archivo CSV
    full = pd.merge(left= file, right = new_file, how="outer" )

    full.to_csv(f'duplicatedDatasets/Duplicated{num_matches}MeanPastMatches.csv', index=False)